In [1]:
import Data.List
import Database.HDBC.Sqlite3
import Database.HDBC
import Numeric.LinearAlgebra.Data
import Numeric.LinearAlgebra.HMatrix

In [2]:
db <- connectSqlite3 "../data/movies.sqlite3"

In [3]:
matrix 3 [1..15]

(5><3)
 [  1.0,  2.0,  3.0
 ,  4.0,  5.0,  6.0
 ,  7.0,  8.0,  9.0
 , 10.0, 11.0, 12.0
 , 13.0, 14.0, 15.0 ]

In [4]:
matrix 5 [1..15]

(3><5)
 [  1.0,  2.0,  3.0,  4.0,  5.0
 ,  6.0,  7.0,  8.0,  9.0, 10.0
 , 11.0, 12.0, 13.0, 14.0, 15.0 ]

In [5]:
matrix 1 [1..5]

(5><1)
 [ 1.0
 , 2.0
 , 3.0
 , 4.0
 , 5.0 ]

In [6]:
matrix 5 [1..5]

(1><5)
 [ 1.0, 2.0, 3.0, 4.0, 5.0 ]

In [7]:
readColumn = map fromSql

In [8]:
raw <- quickQuery db "SELECT action, adventure, animation, childrens, comedy, crime, documentary, drama, fantasy, filmnoir, horror, musical, mystery, romance, scifi, thriller, war, western FROM items ORDER BY movieid" []

In [9]:
full = map readColumn raw :: [[Double]]

In [10]:
head full

[0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]

In [11]:
length full

1682

In [12]:
length (head full)

18

In [13]:
m = fromLists full

In [14]:
1682*18

30276

In [15]:
dependentRaw <- quickQuery db "SELECT avg(rating) FROM data GROUP BY itemid ORDER BY itemid" []

In [16]:
length dependentRaw

1682

In [17]:
dependent = readColumn $ head (transpose dependentRaw) :: [Double]

In [18]:
d = matrix 1 dependent

In [19]:
m = fromLists (map (\row -> row ++ [1]) full)

In [20]:
p = mul (inv (mul (tr m) m)) (mul (tr m) d)

In [21]:
p

(19><1)
 [   -0.1791092688282312
 ,   0.30419457048844745
 ,   0.45069819239411224
 ,   -0.2635577207662525
 , 5.9343374373696633e-2
 ,   0.15329428945699153
 ,    0.3866907794187533
 ,    0.2854049673559267
 , -8.285258338404444e-2
 ,   0.44845610841716343
 ,  -0.17457492721650425
 ,   0.37397995547966545
 ,    0.2764134961019167
 ,   0.21437588668728935
 ,   0.21134308369784005
 ,   0.15293274247923172
 ,    0.4204185730622596
 ,   0.31830357734400394
 ,     2.815862777685112 ]

In [22]:
zip (words "action, adventure, animation, childrens, comedy, crime, documentary, drama, fantasy, filmnoir, horror, musical, mystery, romance, scifi, thriller, war, western, intercept") ((concat . toLists) p)

[("action,",-0.1791092688282312),("adventure,",0.30419457048844745),("animation,",0.45069819239411224),("childrens,",-0.2635577207662525),("comedy,",5.9343374373696633e-2),("crime,",0.15329428945699153),("documentary,",0.3866907794187533),("drama,",0.2854049673559267),("fantasy,",-8.285258338404444e-2),("filmnoir,",0.44845610841716343),("horror,",-0.17457492721650425),("musical,",0.37397995547966545),("mystery,",0.2764134961019167),("romance,",0.21437588668728935),("scifi,",0.21134308369784005),("thriller,",0.15293274247923172),("war,",0.4204185730622596),("western,",0.31830357734400394),("intercept",2.815862777685112)]

In [23]:
fillAll rows cols val = matrix cols (replicate (rows*cols) val)

In [24]:
fillAll 3 4 10

(3><4)
 [ 10.0, 10.0, 10.0, 10.0
 , 10.0, 10.0, 10.0, 10.0
 , 10.0, 10.0, 10.0, 10.0 ]

In [25]:
ybar = sum (head (transpose (toLists d))) / 1682

In [26]:
ybar

3.0760445083251824

In [27]:
syy = mul (tr (d - fillAll 1682 1 ybar)) (d - fillAll 1682 1 ybar)

In [28]:
syy

(1><1)
 [ 1027.082175220263 ]

In [29]:
se = mul (tr (d - mul m p)) (d - mul m p)

In [30]:
se

(1><1)
 [ 935.8935078508722 ]

In [31]:
rsqrd = atIndex (1 - se / syy) (0, 0)

In [32]:
rsqrd

8.878419815808303e-2

In [33]:
rsqrdadj = 1 - (1 - rsqrd) * (1682 - 1) / (1682-18-1)

In [34]:
rsqrdadj

7.892136927464677e-2

In [36]:

adjrsqrd :: Matrix Double -> Matrix Double -> Double
adjrsqrd m d = 1 - (1-rsqrd) * (n-1) / (n-covars-1)
  where
    p = mul (inv (mul (tr m) m)) (mul (tr m) d)
    ybar = sum (head (transpose (toLists d))) / n
    ybarss = mul (tr (d - fillAll n 1 ybar)) (d - fillAll n 1 ybar)
    yhatss = mul (tr (mul m p - d)) (mul m p - d)
    rsqrd = atIndex (1 - yhatss / ybarss) (0,0)
    n = fromIntegral (fst (size m))
    covars = fromIntegral (snd (size m) - 1)
    fillAll rows cols val = matrix cols (replicate (rows*cols) val)

In [37]:
adjrsqrd m d

7.892136927464677e-2

In [38]:
map (\n -> (n, adjrsqrd (m ¿ ([0..(n-1)]++[(n+1)..18])) d)) [0..17]

[(0,7.454695938799527e-2),(1,7.115937576141262e-2),(2,7.355893950274739e-2),(3,7.45841243623846e-2),(4,7.869068670053092e-2),(5,7.727914829954108e-2),(6,7.340225595005156e-2),(7,6.086866115910661e-2),(8,7.934468625313207e-2),(9,7.523268482191314e-2),(10,7.721611895326652e-2),(11,7.303451845888742e-2),(12,7.53755524179055e-2),(13,7.053018893200913e-2),(14,7.592424835664491e-2),(15,7.567252614589393e-2),(16,6.800788528807522e-2),(17,7.691843246322227e-2)]

In [39]:
map (\n -> (n, 7.892136927464677e-2 < adjrsqrd (m ¿ ([0..(n-1)]++[(n+1)..18])) d)) [0..17]

[(0,False),(1,False),(2,False),(3,False),(4,False),(5,False),(6,False),(7,False),(8,True),(9,False),(10,False),(11,False),(12,False),(13,False),(14,False),(15,False),(16,False),(17,False)]

In [41]:
adjrsqrd (m ¿ [0,1,2,3,4,5,6,7,9,10,11,12,13,14,15,16,17,18]) d

7.934468625313207e-2